In [1]:
%load_ext autoreload
%autoreload 2

In [73]:
import sys
from pathlib import Path
from autoeq.constants import PEQ_CONFIGS
from autoeq.batch_processing import batch_processing
ROOT_PATH = Path().resolve()
import ipywidgets as widgets
from IPython.display import display
from measurements.rtings.rtings_crawler import RtingsCrawler
from measurements.crinacle.crinacle_crawler import CrinacleCrawler
from measurements.oratory1990.oratory1990_crawler import Oratory1990Crawler
from measurements.average import average_measurements
from measurements.rename_measurements import rename_measurements
from results.prune_results import prune_results
from results.update_indexes import update_all_indexes
from webapp.create_data import write_entries_and_measurements

## Crawling and Parsing
Additional Python packages are required for processing the measurements:
```bash
python -m pip install -U -r measurements/dev-requirements.txt
```

Measurement crawlers require [Google Chrome](https://www.google.com/chrome/) installed and
[ChromeDriver](https://googlechromelabs.github.io/chrome-for-testing/) binary in the measurements folder (or anywhere
in the PATH).

Measurement crawlers also require C++. This should be installed by default on Linux but on Windows you need to install
Microsoft Visual Studio build tools for this. https://visualstudio.microsoft.com/downloads/ ->
"Tools for Visual Studio 2019" -> "Build Tools for Visual Studio 2019".

oratory1990 crawler requires Ghostscript installed: https://www.ghostscript.com/download/gsdnld.html

### Crinacle
Download measurement data from Drive folder to `measurements/crinacle/raw_data/` before running this!

* `IEM Measurements/IEC60318-4 IEM Measurements (TSV txt)` into `AutoEq/measurements/crinacle/raw_data/IEC60318-4 IEM Measurements (TSV txt)`
* `IEM Measurements/4620 IEM Measurements` into `AutoEq/measurements/crinacle/raw_data/4620 IEM Measurements`
* `HP Measurements/EARS + 711 (TSV txt) (Legacy)` into `AutoEq/measurements/crinacle/raw_data/EARS + 711 (TSV txt) (Legacy)`
* `GRAS 43AG-7` into `AutoEq/measurements/crinacle/raw_data/GRAS 43AG-7`

In [76]:
crawler = CrinacleCrawler()
crawler.run()
display(crawler.widget)

### oratory1990
oratory1990 crawler fetches all measurements from https://www.reddit.com/r/oratory1990/wiki/index/list_of_presets/, downloads PDFs and reads the frequency response measurement data from the PDFs. Parsing the PDFs requires [Ghostscript](https://www.ghostscript.com/download/gsdnld.html) to be installed on the system.

In [7]:
crawler = Oratory1990Crawler()
crawler.process(new_only=True)
display(crawler.widget)

### Rtings
Rtings crawler fetches all measurements from https://www.rtings.com/headphones/1-5/graph and downloads and parses the JSON data files.

In [ ]:
crawler = RtingsCrawler()
crawler.process()
display(crawler.widget)

## Rename Measurements
Sometimes measurements are named incorrectly or previously only one sample existed and now multiple samples have been measured and so the original one needs to be renamed as "<name> (sample 1)"

In [ ]:
renames = [
    {'old_name': '', 'new_name': '', 'dbs': ['']},
]
rename_measurements(renames, dry_run=False)

## Prune Results
Check if obsolete results (e.g. because of renaming) exist and remove them

In [87]:
prune_results(databases=['crinacle'], dry_run=False)

Removed "crinacle\711 in-ear\64 Audio A18s"
Removed "crinacle\711 in-ear\64 Audio A18s (sample 1)"
Removed "crinacle\711 in-ear\64 Audio A18s (sample 2)"
Removed "crinacle\711 in-ear\64 Audio N8 (CIEM)"
Removed "crinacle\711 in-ear\64 Audio N8 (CIEM) (sample 1)"
Removed "crinacle\711 in-ear\64 Audio N8 (CIEM) (sample 2)"
Removed "crinacle\711 in-ear\64 Audio N8 (CIEM) (sample 3)"
Removed "crinacle\711 in-ear\64 Audio N8 (CIEM) (sample 6)"
Removed "crinacle\711 in-ear\64 Audio N8 (m15 Apex module)"
Removed "crinacle\711 in-ear\64 Audio N8 (m15 Apex module) (CIEM)"
Removed "crinacle\711 in-ear\64 Audio N8 (m15 Apex module) (CIEM) (sample 2)"
Removed "crinacle\711 in-ear\64 Audio N8 (m15 Apex module) (CIEM) (sample 3)"
Removed "crinacle\711 in-ear\64 Audio N8 (m15 Apex module) (CIEM) (sample 4)"
Removed "crinacle\711 in-ear\64 Audio N8 (m15 Apex module) (CIEM) (sample 5)"
Removed "crinacle\711 in-ear\64 Audio N8 (m15 Apex module) (sample 1)"
Removed "crinacle\711 in-ear\64 Audio N8 (m15 A

## Update Results
Creates new results from the measurements. `eq_kwargs` are parameters shared by all jobs.

In [5]:
eq_kwargs = {
    'parametric_eq': True, 'ten_band_eq': True, 'convolution_eq': True,
    'parametric_eq_config': [PEQ_CONFIGS['4_PEAKING_WITH_LOW_SHELF'], PEQ_CONFIGS['4_PEAKING_WITH_HIGH_SHELF']],
    'fs': [44100, 48000],
    'thread_count': 0,
}

#### oratory1990 Over-ear

In [5]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'oratory1990', 'data', 'over-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'oratory1990', 'over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/1 [00:00<?, ?it/s]

#### oratory1990 In-ear

In [98]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'oratory1990', 'data', 'in-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'oratory1990', 'in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'autoeq_in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/5 [00:00<?, ?it/s]

#### oratory1990 Earbud

In [6]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'oratory1990', 'data', 'earbud'),
    output_dir=ROOT_PATH.joinpath('results', 'oratory1990', 'earbud'),
    compensation=ROOT_PATH.joinpath('compensation', 'autoeq_in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### crinacle GRAS 43AG-7 On-ear

In [100]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'over-ear', 'GRAS 43AG-7'),
    output_dir=ROOT_PATH.joinpath('results', 'crinacle', 'GRAS 43AG-7 over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/49 [00:00<?, ?it/s]

#### crinacle EARS+711 Over-ear

In [101]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'over-ear', 'EARS + 711'),
    output_dir=ROOT_PATH.joinpath('results', 'crinacle', 'EARS + 711 over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'crinacle_harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/2 [00:00<?, ?it/s]

#### crinacle 4620 In-ear

In [102]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'in-ear', 'Bruel & Kjaer 4620'),
    output_dir=ROOT_PATH.joinpath('results', 'crinacle', 'Bruel & Kjaer 4620 in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'diffuse_field_5128_-1dBpoct.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/8 [00:00<?, ?it/s]

#### crinacle 711 In-ear

In [103]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'in-ear', '711'),
    output_dir=ROOT_PATH.joinpath('results', 'crinacle', '711 in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'autoeq_in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/109 [00:00<?, ?it/s]

#### Rtings Over-ear

In [9]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'rtings', 'data', 'over-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Rtings', 'over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'rtings_harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/2 [00:00<?, ?it/s]

#### Rtings In-ear

In [10]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'rtings', 'data', 'in-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Rtings', 'in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'rtings_autoeq_in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Rtings Earbud

In [11]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'rtings', 'data', 'earbud'),
    output_dir=ROOT_PATH.joinpath('results', 'Rtings', 'earbud'),
    compensation=ROOT_PATH.joinpath('compensation', 'rtings_autoeq_in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Innerfidelity In-ear

In [111]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'innerfidelity', 'data', 'over-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Innerfidelity', 'over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'innerfidelity_harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/14 [00:00<?, ?it/s]

#### Innerfidelity In-ear

In [112]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'innerfidelity', 'data', 'in-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Innerfidelity', 'in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'innerfidelity_autoeq_in-ear.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/10 [00:00<?, ?it/s]

#### Innerfidelity Earbud

In [113]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'innerfidelity', 'data', 'earbud'),
    output_dir=ROOT_PATH.joinpath('results', 'Innerfidelity', 'earbud'),
    compensation=ROOT_PATH.joinpath('compensation', 'innerfidelity_autoeq_in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Headphone.com Legacy Over-ear

In [114]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'headphonecom', 'data', 'over-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Headphone.com Legacy', 'over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'headphonecom_harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/12 [00:00<?, ?it/s]

#### Headphone.com Legacy In-ear

In [115]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'headphonecom', 'data', 'in-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Headphone.com Legacy', 'in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'headphonecom_autoeq_in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Headphone.com Legacy Earbud

In [116]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'headphonecom', 'data', 'earbud'),
    output_dir=ROOT_PATH.joinpath('results', 'Headphone.com Legacy', 'earbud'),
    compensation=ROOT_PATH.joinpath('compensation', 'headphonecom_autoeq_in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/10 [00:00<?, ?it/s]

## Update Indexes
Updates recommended results, full results, DB specific results, HeSuVi results and ranking table.

In [3]:
update_all_indexes()

Creating ranking index...


  0%|          | 0/3649 [00:00<?, ?it/s]

Creating recommendations index...
Creating full index...
Creating source indices...
Creating HeSuVi ZIP archive...


  0%|          | 0/3649 [00:00<?, ?it/s]

#### Data for webapp

In [ ]:
write_entries_and_measurements()

## Deploy
1. Add files to Git, commit and push
2. Upload webapp data to server

# Sandbox
Don't run these! Random exploration while developing.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from measurements.name_index import NameItem, NameIndex
from measurements.crinacle.crinacle_crawler import CrinacleCrawler

#### Name changes:
-UE18+ 3rd Gen --> (3rd Gen)

In [64]:
crawler = CrinacleCrawler()

In [67]:
crawler.crawl()
crawler.create_prompts()
crawler.reload_ui()
display(crawler.widget)

In [ ]:
crawler.process(new_only=True)

### Remove Samples from Name Index
Samples will be averaged during processing, not afterwards and therefore need to be removed from the index

In [41]:
items = [item for item in crawler.name_index.items]
for item in items:
    if item.name is not None and '(sample' in item.name:
        crawler.name_index.remove(item)

In [42]:
crawler.write_name_index()